# Imports

In [1]:
import subprocess
import json
from pathlib import Path
from glob2 import glob
import os
from multiprocessing import Pool
from datetime import datetime
from uuid import uuid4

# Functions

In [2]:
def runFullParse (inputPiece, iterations=10):
    (corpus, piece) = inputPiece
    jsonPath = "outputs/"+corpus+"/"+piece+".json"
    cmd = ["stack","run","fullParse","--","-n",str(iterations), corpus, piece, "All"] 
    print("Running command: " + (" ".join(cmd)))
    print("Expecting results in " + jsonPath)
    
    #res = subprocess.run(cmd, cwd="..")
    #res = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd="..")
    res = subprocess.run(cmd,stderr=subprocess.PIPE, cwd="..")

    if res.returncode != 0:
        print("Error in subprocess:")
        print(res.stderr)
        return;
    else:
        f = open(jsonPath)
        results = json.load(f)
        f.close()
        return results
     
def get_corpus_pieces(corpus):
    return sorted (os.path.basename(d).split(".")[0] for d in glob("inputs/slices/"+corpus+"/*.csv"))

def get_corpi():
    return [os.path.basename(d) for d in glob("inputs/slices/*")][1:]

def run_experiment():
    corpi = get_corpi()
    for corpus in corpi:
        pieces = get_corpus_pieces(corpus)
        for piece in pieces:
            runFullParse((corpus, piece))

In [3]:
def run_Experiment(commands,name,threads):
    id = datetime.now().strftime('%Y%m-%d%H-%M%S_') + name

    commands = [ ["stack","run","fullParse","--","-n",str(1), "-id", str(id)] + command for command in commands]
    with Pool(threads) as p:
        p.map(run_command, commands)


In [4]:
def run_command(command):
    print("\nRunning command: " + (" ".join(command)))    
    #res = subprocess.run(cmd, cwd="..")
    #res = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd="..")
    res = subprocess.run(command,stderr=subprocess.PIPE, cwd="..")

    if res.returncode != 0:
        print("Error in subprocess:")
        print(res.stderr)
        return;


In [5]:
def full_experiment(threads=20):
    commands = []
    corpi = get_corpi()
    for corpus in corpi:
        pieces = get_corpus_pieces(corpus)
        for piece in pieces:
            for algorithm in ["RandomWalk", "RandomSample", "RandomReduction","BeamSearch 10"]:
                commands.append([ corpus, piece, algorithm])

    run_Experiment(commands, "baseline", threads)

In [6]:
def baseline_experiment(threads=20):
    commands = []
    corpi = get_corpi()
    for corpus in corpi:
        pieces = get_corpus_pieces(corpus)
        for piece in pieces:
            for algorithm in ["RandomWalk", "RandomSample", "RandomReduction"]:
                commands.append([ corpus, piece, algorithm])

    run_Experiment(commands, "baseline", threads)


In [7]:
def beamwidth_experiment(threads=10):
    commands = []
    corpi = get_corpi()
    for corpus in corpi:
        pieces = get_corpus_pieces(corpus)
        for piece in pieces:
            for w in range (1,20):
                algo = "BeamSearch " + str(w)
                commands.append([ corpus, piece, algo])

    run_Experiment(commands, "beamwidth", threads)

In [8]:
print("\"io")

"io


# Run Experiments

In [ ]:
if __name__ == '__main__':
    #algo_experiment()
    beamwidth_experiment()


Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op12n07 BeamSearch 15
Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op12n01 BeamSearch 1

Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op12n04 BeamSearch 8
Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op38n03 BeamSearch 3
Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op38n06 BeamSearch 10
Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op43n01 BeamSearch 17
Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op43n05 BeamSearch 5
Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op47n02 BeamSearch 12

Running command: stack run fullParse -- -n 1 -id 202304-1617-4112

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)






Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op12n07 BeamSearch 16
Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op38n03 BeamSearch 6

Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op12n01 BeamSearch 5

Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op38n03 BeamSearch 7

Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op12n01 BeamSearch 6

Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op38n03 BeamSearch 8

Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op12n01 BeamSearch 7
Error in subprocess:


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)






Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op12n07 BeamSearch 17
Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op38n03 BeamSearch 9

Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op12n01 BeamSearch 8

Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op12n01 BeamSearch 9

Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op38n03 BeamSearch 10

Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op12n01 BeamSearch 10

Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op38n03 BeamSearch 11
Error in subprocess:


...




Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op43n02 BeamSearch 3

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)






Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op12n04 BeamSearch 10
Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op12n08 BeamSearch 11

Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op12n08 BeamSearch 12
Error in subprocess:

Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op12n08 BeamSearch 13


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)





Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op38n04 BeamSearch 3

Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op12n08 BeamSearch 14

Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op12n02 BeamSearch 2

Running command: stack run fullParse -- -n 1 -id 202304-1617-4112_beamwidth grieg_lyric_pieces op12n08 BeamSearch 15


In [ ]:
if __name__ == '__main__':
    algo_experiment()
    #beamwidth_experiment()